In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-19 17:52:40.076863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 17:52:40.153306: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 17:52:40.488152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 240
WIDTH = 320


SHOW_PREVIEW = True

FIXED_DELTA_SECONDS = 0.2

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    discrete_actions = {0: [0, 0.5]}
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.actor_list = []

        self.action_space = spaces.Discrete(len(self.discrete_actions))

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)

        self.client.load_world('Town01')

        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        self.settings.no_rendering_mode = False

        self.world.apply_settings(self.settings)

        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.cleanup()
        
        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        
        self.transform = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.transform.location
        self.point_b = self.spawn_points[241].location
        
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.transform)
        
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()
            return self.reset()
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")

        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z, x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2)  # not detect a collision when the car spawns/falls from sky.

        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     
        while self.front_camera is None:
            time.sleep(0.01)  # Just in case car takes any longer.

        self.episode_start = time.time()
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        # Show camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
        
    def step(self, action):
        self.step_counter += 1
        steer, throttle = self.discrete_actions[action]

        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=0.0))

        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        print(f"Current speed: {kmh} km/h, Distance to target: {distance_to_destination} meters")

        # Decelerate as the vehicle approaches the target(Smooth Stopping)
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        if distance_to_destination < 50.0:  # Within 10 meters of the target
            throttle = max(0.1, throttle * (distance_to_destination / 50.0))  # never below 0.1 3shan mto2afsh 5ales abl ma nwsl
            

        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=0.0))


        
        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # Initialize variables
        done = False
        terminated = False
        truncated = False
        reward = 0

        # Check for collision
        if len(self.collision_hist) != 0:
            print("Collision occurred")
            reward = -5
            done = True
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            observation = self.reset()[0]
            return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

        
        
        # Check if the car has reached the goal point
        if distance_to_destination < 10.0:  # Threshold for reaching the destination
            print("Goal reached")
            reward = 5
            done = True
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            observation = self.reset()[0]
            return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

        # Check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            print("Time limit exceeded")
            done = True
            truncated = True
            self.cleanup()
            observation = self.front_camera
            return observation.astype(np.uint8), 0, terminated, truncated, {}  # Return an empty dictionary as info

        # Calculate reward based on distance traveled towards point B
        reward = 1 / distance_to_destination  # Inverse proportional to distance

        # Get observation
        observation = self.front_camera

        return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        cv2.destroyAllWindows()

    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3]  # Ignore the 4th Alpha channel - up to 3
        self.front_camera = i / 255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:
'''
env = CarEnv()

check_env(env)
'''

'\nenv = CarEnv()\n\ncheck_env(env)\n'

In [5]:
model_name = "one_throttle_one_steer"
models_dir = f"models/{model_name}/"

logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001, tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1716130362/PPO_0
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
Current 

Current speed: 4 km/h, Distance to target: 179.87840270996094 meters
Current speed: 4 km/h, Distance to target: 179.86917114257812 meters
Current speed: 4 km/h, Distance to target: 179.86917114257812 meters
Current speed: 4 km/h, Distance to target: 179.85704040527344 meters
Current speed: 4 km/h, Distance to target: 179.85704040527344 meters
Current speed: 4 km/h, Distance to target: 179.85704040527344 meters
Current speed: 5 km/h, Distance to target: 179.8414764404297 meters
Current speed: 5 km/h, Distance to target: 179.83045959472656 meters
Current speed: 5 km/h, Distance to target: 179.83045959472656 meters
Current speed: 6 km/h, Distance to target: 179.73809814453125 meters
Current speed: 6 km/h, Distance to target: 179.73809814453125 meters
Current speed: 7 km/h, Distance to target: 179.7238006591797 meters
Current speed: 7 km/h, Distance to target: 179.7103729248047 meters
Current speed: 7 km/h, Distance to target: 179.7103729248047 meters
Current speed: 7 km/h, Distance to tar

Current speed: 14 km/h, Distance to target: 177.4044952392578 meters
Current speed: 14 km/h, Distance to target: 177.3726806640625 meters
Current speed: 15 km/h, Distance to target: 177.34373474121094 meters
Current speed: 15 km/h, Distance to target: 177.31198120117188 meters
Current speed: 15 km/h, Distance to target: 177.31198120117188 meters
Current speed: 15 km/h, Distance to target: 177.28195190429688 meters
Current speed: 15 km/h, Distance to target: 177.2509765625 meters
Current speed: 15 km/h, Distance to target: 177.22190856933594 meters
Current speed: 15 km/h, Distance to target: 177.18955993652344 meters
Current speed: 15 km/h, Distance to target: 177.15965270996094 meters
Current speed: 15 km/h, Distance to target: 177.13201904296875 meters
Current speed: 15 km/h, Distance to target: 177.13201904296875 meters
Current speed: 15 km/h, Distance to target: 177.13201904296875 meters
Current speed: 15 km/h, Distance to target: 177.08404541015625 meters
Current speed: 15 km/h, Di

Current speed: 20 km/h, Distance to target: 173.36886596679688 meters
Current speed: 20 km/h, Distance to target: 173.325439453125 meters
Current speed: 20 km/h, Distance to target: 173.325439453125 meters
Current speed: 20 km/h, Distance to target: 173.28396606445312 meters
Current speed: 20 km/h, Distance to target: 173.24465942382812 meters
Current speed: 20 km/h, Distance to target: 173.20034790039062 meters
Current speed: 20 km/h, Distance to target: 173.20034790039062 meters
Current speed: 20 km/h, Distance to target: 173.15567016601562 meters
Current speed: 20 km/h, Distance to target: 173.11361694335938 meters
Current speed: 20 km/h, Distance to target: 173.11361694335938 meters
Current speed: 20 km/h, Distance to target: 173.06649780273438 meters
Current speed: 20 km/h, Distance to target: 173.02105712890625 meters
Current speed: 20 km/h, Distance to target: 172.98086547851562 meters
Current speed: 20 km/h, Distance to target: 172.9383544921875 meters
Current speed: 20 km/h, D

Current speed: 25 km/h, Distance to target: 168.19998168945312 meters
Current speed: 25 km/h, Distance to target: 168.19998168945312 meters
Current speed: 25 km/h, Distance to target: 168.1492156982422 meters
Current speed: 25 km/h, Distance to target: 168.09063720703125 meters
Current speed: 25 km/h, Distance to target: 168.09063720703125 meters
Current speed: 25 km/h, Distance to target: 168.03294372558594 meters
Current speed: 25 km/h, Distance to target: 167.98348999023438 meters
Current speed: 25 km/h, Distance to target: 167.9351348876953 meters
Current speed: 25 km/h, Distance to target: 167.87889099121094 meters
Current speed: 25 km/h, Distance to target: 167.87889099121094 meters
Current speed: 25 km/h, Distance to target: 167.8222198486328 meters
Current speed: 25 km/h, Distance to target: 167.77349853515625 meters
Current speed: 25 km/h, Distance to target: 167.77349853515625 meters
Current speed: 25 km/h, Distance to target: 167.7184600830078 meters
Current speed: 25 km/h, 

...

Current speed: 11 km/h, Distance to target: 178.87693786621094 meters
Current speed: 11 km/h, Distance to target: 178.87693786621094 meters
Current speed: 11 km/h, Distance to target: 178.84608459472656 meters
Current speed: 11 km/h, Distance to target: 178.82330322265625 meters
Current speed: 11 km/h, Distance to target: 178.80160522460938 meters
Current speed: 11 km/h, Distance to target: 178.80160522460938 meters
Current speed: 11 km/h, Distance to target: 178.7668914794922 meters
Current speed: 11 km/h, Distance to target: 178.74057006835938 meters
Current speed: 11 km/h, Distance to target: 178.74057006835938 meters
Current speed: 11 km/h, Distance to target: 178.71670532226562 meters
Current speed: 11 km/h, Distance to target: 178.6935272216797 meters
Current speed: 11 km/h, Distance to target: 178.6642608642578 meters
Current speed: 11 km/h, Distance to target: 178.6642608642578 meters
Current speed: 12 km/h, Distance to target: 178.63674926757812 meters
Current speed: 12 km/h, 

Current speed: 18 km/h, Distance to target: 175.516845703125 meters
Current speed: 18 km/h, Distance to target: 175.4820556640625 meters
Current speed: 18 km/h, Distance to target: 175.4420166015625 meters
Current speed: 18 km/h, Distance to target: 175.40591430664062 meters
Current speed: 18 km/h, Distance to target: 175.40591430664062 meters
Current speed: 18 km/h, Distance to target: 175.3675537109375 meters
Current speed: 18 km/h, Distance to target: 175.33160400390625 meters
Current speed: 18 km/h, Distance to target: 175.29342651367188 meters
Current speed: 18 km/h, Distance to target: 175.25473022460938 meters
Current speed: 18 km/h, Distance to target: 175.21780395507812 meters
Current speed: 18 km/h, Distance to target: 175.18341064453125 meters
Current speed: 18 km/h, Distance to target: 175.1500244140625 meters
Current speed: 18 km/h, Distance to target: 175.11373901367188 meters
Current speed: 18 km/h, Distance to target: 175.07635498046875 meters
Current speed: 18 km/h, Di

Current speed: 22 km/h, Distance to target: 170.87911987304688 meters
Current speed: 22 km/h, Distance to target: 170.87911987304688 meters
Current speed: 22 km/h, Distance to target: 170.82839965820312 meters
Current speed: 22 km/h, Distance to target: 170.78192138671875 meters
Current speed: 23 km/h, Distance to target: 170.7291259765625 meters
Current speed: 23 km/h, Distance to target: 170.7291259765625 meters
Current speed: 22 km/h, Distance to target: 170.6868896484375 meters
Current speed: 23 km/h, Distance to target: 170.63726806640625 meters
Current speed: 23 km/h, Distance to target: 170.58773803710938 meters
Current speed: 23 km/h, Distance to target: 170.58773803710938 meters
Current speed: 23 km/h, Distance to target: 170.54177856445312 meters
Current speed: 23 km/h, Distance to target: 170.4935302734375 meters
Current speed: 23 km/h, Distance to target: 170.2774658203125 meters
Current speed: 23 km/h, Distance to target: 170.16238403320312 meters
Current speed: 23 km/h, D

Current speed: 27 km/h, Distance to target: 165.1361541748047 meters
Current speed: 27 km/h, Distance to target: 165.0832977294922 meters
Current speed: 27 km/h, Distance to target: 165.0832977294922 meters
Current speed: 27 km/h, Distance to target: 165.0234832763672 meters
Current speed: 27 km/h, Distance to target: 165.0234832763672 meters
Current speed: 27 km/h, Distance to target: 164.94227600097656 meters
Current speed: 28 km/h, Distance to target: 164.8815155029297 meters
Current speed: 28 km/h, Distance to target: 164.8815155029297 meters
Current speed: 28 km/h, Distance to target: 164.81105041503906 meters
Current speed: 28 km/h, Distance to target: 164.7239532470703 meters
Current speed: 28 km/h, Distance to target: 164.66456604003906 meters
Current speed: 28 km/h, Distance to target: 164.66456604003906 meters
Current speed: 28 km/h, Distance to target: 164.59437561035156 meters
Current speed: 28 km/h, Distance to target: 164.5276336669922 meters
Current speed: 28 km/h, Dista

KeyboardInterrupt: 

In [ ]:
'''
discrete_actions = {
    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}
Action 0: Steer left (-1) and throttle (1)
Action 1: Do not steer (0) and throttle (1)
Action 2: Steer right (1) and throttle (1)
Action 3: Do not steer (0) and do not throttle (0)
'''

In [ ]:
#discrete_actions = {
 #   0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}

In [ ]:
#len(discrete_actions)